In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%time
! pip install --upgrade --force-reinstall --no-deps  kaggle > /dev/null
! mkdir ~/.kaggle

! cp "/content/drive/My Drive/Kaggle/kaggle.json" ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

import os

if not os.path.exists("/content/input/"):
    !mkdir input
    !kaggle competitions download -c birdclef-2022
    #!unzip birdclef-2022.zip -d input/birdclef-2022
    
    n_files = !unzip -l birdclef-2022.zip | grep .pt | wc -l
    !unzip -o birdclef-2022.zip -d /home/data/ | pv -l -s {n_files[0]} > /dev/null

100% 6.11G/6.12G [01:20<00:00, 104MB/s] 
100% 6.12G/6.12G [01:20<00:00, 81.2MB/s]
/bin/bash: pv: command not found
CPU times: user 849 ms, sys: 197 ms, total: 1.05 s
Wall time: 1min 26s


In [ ]:
!mkdir working

In [ ]:
cd working

/content/working


In [ ]:
!ls input/birdclef-2022

ls: cannot access 'input/birdclef-2022': No such file or directory


In [ ]:
!pip show torchaudio

Name: torchaudio
Version: 0.10.0+cu111
Summary: An audio package for PyTorch
Home-page: https://github.com/pytorch/audio
Author: Soumith Chintala, David Pollack, Sean Naren, Peter Goldsborough
Author-email: soumith@pytorch.org
License: UNKNOWN
Location: /usr/local/lib/python3.7/dist-packages
Requires: torch
Required-by: 


In [ ]:
import os
import random
import numpy as np
import pandas as pd
import librosa
import librosa.display
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn import model_selection
from sklearn import preprocessing
import IPython.display as ipd

import IPython.display
from IPython.display import display

import requests
import matplotlib.pyplot as plt

import torch
import torchaudio
import torchaudio.functional as F
import torchaudio.transforms as T

import soundfile as sf
from  soundfile import SoundFile
import numpy as np

from tqdm.notebook import tqdm
from pathlib import Path
import joblib, json
from  sklearn.model_selection  import StratifiedKFold

In [ ]:
torch.multiprocessing.set_start_method('spawn')

In [ ]:
TRAIN_AUDIO_IMAGES_SAVE_ROOT = Path("/content/output")

In [ ]:
!python /content/drive/MyDrive/python/kaggle/birdclef-2022/code/get_melspec_torch.py

cuda
  0%|          | 0/14852 [00:00<?, ?it/s]
/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:705: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [ ]:
cd $TRAIN_AUDIO_IMAGES_SAVE_ROOT

/content/output


In [ ]:
#n_files = !unzip -l /content/working/melspcec.zip | grep .pt | wc -l

In [ ]:


def make_df(n_splits=5, seed=42, nrows=None):
    
    df = pd.read_csv(DATA_ROOT/"train_metadata.csv", nrows=nrows)

    LABEL_IDS = {label: label_id for label_id,label in enumerate(sorted(df["primary_label"].unique()))}
    
    #df = df.iloc[PART_INDEXES[PART_ID]: PART_INDEXES[PART_ID+1]]

    df["label_id"] = df["primary_label"].map(LABEL_IDS)

    df["filepath"] = [str(TRAIN_AUDIO_ROOT/filename) for filename in df["filename"].values ]

    pool = joblib.Parallel(4)
    mapper = joblib.delayed(get_audio_info)
    tasks = [mapper(filepath) for filepath in df.filepath]

    df = pd.concat([df, pd.DataFrame(pool(tqdm(tasks)))], axis=1, sort=False)
    
    skf = StratifiedKFold(n_splits=n_splits, random_state=seed, shuffle=True)
    splits = skf.split(np.arange(len(df)), y=df.label_id.values)
    df["fold"] = -1

    for fold, (train_set, val_set) in enumerate(splits):
        
        df.loc[df.index[val_set], "fold"] = fold

    return LABEL_IDS, df

In [ ]:
    LABEL_IDS, df = make_df(nrows=None, SEED=SEED)
    df.to_csv(MYDRIVE/"df_metadata.csv",index=False)


        with open(MYDRIVE/"LABEL_IDS.json", "w") as f:
            json.dump(LABEL_IDS, f)

In [ ]:
!zip -r /content/working/melspcec.zip ./*

ストリーミング出力は最後の 5000 行に切り捨てられました。
  adding: normoc/XC122159.pt (deflated 78%)
  adding: normoc/XC638306.pt (deflated 94%)
  adding: normoc/XC534762.pt (deflated 90%)
  adding: normoc/XC371812.pt (deflated 88%)
  adding: normoc/XC355513.pt (deflated 90%)
  adding: normoc/XC369853.pt (deflated 83%)
  adding: normoc/XC162118.pt (deflated 76%)
  adding: normoc/XC602014.pt (deflated 90%)
  adding: norpin/ (stored 0%)
  adding: norpin/XC266784.pt (deflated 75%)
  adding: norpin/XC291860.pt (deflated 94%)
  adding: norpin/XC627557.pt (deflated 90%)
  adding: norpin/XC496284.pt (deflated 70%)
  adding: norpin/XC627633.pt (deflated 78%)
  adding: norpin/XC617858.pt (deflated 88%)
  adding: norpin/XC606093.pt (deflated 77%)
  adding: norpin/XC479156.pt (deflated 81%)
  adding: norpin/XC113161.pt (deflated 89%)
  adding: norpin/XC632559.pt (deflated 74%)
  adding: norpin/XC551679.pt (deflated 94%)
  adding: norpin/XC611504.pt (deflated 91%)
  adding: norpin/XC624461.pt (deflated 83%)
  adding: norp

In [ ]:
!cp melspcec.zip /content/working/

In [ ]:
!ls /content/working/

audio_images  melspcec.zip


In [ ]:

kaggle_savepath = "/content/working"

%cd /content/

!cp "/content/drive/MyDrive/Kaggle/kaggle.json" ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets init -p $kaggle_savepath

import json
from collections import OrderedDict
import re

with open(f'{kaggle_savepath}/dataset-metadata.json') as f:
    df = json.load(f)

expname = "kkiller-base"
datasettitle = "melspec-" + expname
datasetid = re.sub(r"[^a-zA-Z0-9]","",datasettitle)

df["title"] = datasettitle
df["id"] = df["id"].split("/")[0] +"/" +datasetid

with open(f'{kaggle_savepath}/dataset-metadata.json', mode='w') as f:
    json.dump(df, f, indent=2)

!kaggle datasets create -p $kaggle_savepath

/content
Data package template written to: /content/working/dataset-metadata.json
Starting upload for file melspcec.zip
100% 2.01G/2.01G [00:58<00:00, 36.8MB/s]
Upload successful: melspcec.zip (2GB)
Your private Dataset is being created. Please check progress at https://www.kaggle.com/datasets/chiman3se/melspeckkillerbase
